In [9]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('hm5') \
    .getOrCreate()

df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv('/u1/fhv_tripdata_2019-10.csv')

df.show()

24/03/03 14:16:44 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [10]:
spark.version

'3.5.1'

In [11]:
df.head(10)

[Row(dispatching_base_num='B00009', pickup_datetime=datetime.datetime(2019, 10, 1, 0, 23), dropOff_datetime=datetime.datetime(2019, 10, 1, 0, 35), PUlocationID=264, DOlocationID=264, SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00013', pickup_datetime=datetime.datetime(2019, 10, 1, 0, 11, 29), dropOff_datetime=datetime.datetime(2019, 10, 1, 0, 13, 22), PUlocationID=264, DOlocationID=264, SR_Flag=None, Affiliated_base_number='B00013'),
 Row(dispatching_base_num='B00014', pickup_datetime=datetime.datetime(2019, 10, 1, 0, 11, 43), dropOff_datetime=datetime.datetime(2019, 10, 1, 0, 37, 20), PUlocationID=264, DOlocationID=264, SR_Flag=None, Affiliated_base_number='B00014'),
 Row(dispatching_base_num='B00014', pickup_datetime=datetime.datetime(2019, 10, 1, 0, 56, 29), dropOff_datetime=datetime.datetime(2019, 10, 1, 0, 57, 47), PUlocationID=264, DOlocationID=264, SR_Flag=None, Affiliated_base_number='B00014'),
 Row(dispatching_base_num='B00014', pickup_datetime=

In [14]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [22]:
df = df.repartition(6)

In [23]:
df.write.parquet('/tmp/fhv_tripdata_2019-10')

In [24]:
df = spark.read.parquet ('/tmp/fhv_tripdata_2019-10')

In [25]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [43]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|pickup_date|dropoff_date|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+------------+
|              B01606|2019-10-05 07:00:17|2019-10-05 07:02:30|         264|         227|   NULL|                B01606| 2019-10-05|  2019-10-05|
|              B00789|2019-10-08 06:40:10|2019-10-08 07:39:57|         264|         264|   NULL|                B00789| 2019-10-08|  2019-10-08|
|              B00900|2019-10-07 07:40:30|2019-10-07 08:12:23|         264|          82|   NULL|                B00900| 2019-10-07|  2019-10-07|
|              B00419|2019-10-04 11:28:39|2019-10-04 11:33:25|          51|          51|   NULL|                B00419| 2019-10-04

In [56]:
from pyspark.sql import col, to_date

In [49]:
df = df.withColumn("pickup_date", to_date(col("pickup_datetime")))
df = df.withColumn("dropoff_date", to_date(col("dropOff_datetime")))

In [50]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|pickup_date|dropoff_date|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+------------+
|              B01606|2019-10-05 07:00:17|2019-10-05 07:02:30|         264|         227|   NULL|                B01606| 2019-10-05|  2019-10-05|
|              B00789|2019-10-08 06:40:10|2019-10-08 07:39:57|         264|         264|   NULL|                B00789| 2019-10-08|  2019-10-08|
|              B00900|2019-10-07 07:40:30|2019-10-07 08:12:23|         264|          82|   NULL|                B00900| 2019-10-07|  2019-10-07|
|              B00419|2019-10-04 11:28:39|2019-10-04 11:33:25|          51|          51|   NULL|                B00419| 2019-10-04

In [51]:
df.select("pickup_datetime", "pickup_date", "dropoff_datetime", "dropoff_date").show()

+-------------------+-----------+-------------------+------------+
|    pickup_datetime|pickup_date|   dropoff_datetime|dropoff_date|
+-------------------+-----------+-------------------+------------+
|2019-10-05 07:00:17| 2019-10-05|2019-10-05 07:02:30|  2019-10-05|
|2019-10-08 06:40:10| 2019-10-08|2019-10-08 07:39:57|  2019-10-08|
|2019-10-07 07:40:30| 2019-10-07|2019-10-07 08:12:23|  2019-10-07|
|2019-10-04 11:28:39| 2019-10-04|2019-10-04 11:33:25|  2019-10-04|
|2019-10-07 11:22:00| 2019-10-07|2019-10-07 11:27:00|  2019-10-07|
|2019-10-04 13:14:00| 2019-10-04|2019-10-04 13:19:00|  2019-10-04|
|2019-10-06 22:44:10| 2019-10-06|2019-10-06 22:48:21|  2019-10-06|
|2019-10-08 01:13:56| 2019-10-08|2019-10-08 01:28:26|  2019-10-08|
|2019-10-08 07:43:32| 2019-10-08|2019-10-08 07:46:57|  2019-10-08|
|2019-10-07 06:33:00| 2019-10-07|2019-10-07 07:56:00|  2019-10-07|
|2019-10-08 07:38:29| 2019-10-08|2019-10-08 07:59:51|  2019-10-08|
|2019-10-05 18:05:13| 2019-10-05|2019-10-05 18:13:53|  2019-10

In [57]:
oct_15_trips = df.filter(col("pickup_date") == "2019-10-15")

In [58]:
oct_15_trips.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|pickup_date|dropoff_date|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+------------+
|              B01984|2019-10-15 06:40:00|2019-10-15 06:57:00|         264|          92|   NULL|                B01984| 2019-10-15|  2019-10-15|
|              B02784|2019-10-15 10:27:03|2019-10-15 11:12:15|          86|          55|   NULL|                  NULL| 2019-10-15|  2019-10-15|
|              B03016|2019-10-15 14:53:52|2019-10-15 15:03:03|         264|         254|   NULL|                B02788| 2019-10-15|  2019-10-15|
|              B01200|2019-10-15 12:30:14|2019-10-15 13:01:03|         264|          33|   NULL|                B01200| 2019-10-15

In [59]:
num_oct_15_trips = oct_15_trips.count()

In [60]:
print(f"Number of taxi trips on October 15th: {num_oct_15_trips}")

Number of taxi trips on October 15th: 62610


In [61]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropOff_datetime',
 'PUlocationID',
 'DOlocationID',
 'SR_Flag',
 'Affiliated_base_number',
 'pickup_date',
 'dropoff_date']

In [62]:
df \
    .withColumn('duration', df.dropOff_datetime.cast('long') - df.pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2019-10-28|   2272149000|
| 2019-10-11|   2272149000|
| 2019-10-31|    315620787|
| 2019-10-01|    252460901|
| 2019-10-17|     31658400|
+-----------+-------------+



In [65]:
df_zones = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv('/u1/taxi_zone_lookup.csv')

In [69]:
df_zones.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [70]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [74]:
df_zones.write.parquet('/tmp/taxi_zone_lookup', mode='overwrite')
df_zones = spark.read.parquet ('/tmp/taxi_zone_lookup')

In [75]:
# Register dataframes as temporary views
df.createOrReplaceTempView("trips")
df_zones.createOrReplaceTempView("zones")

In [76]:
# Join the dataframes on PUlocationID
joined_df = spark.sql("""
    SELECT t.*, z.Zone
    FROM trips t
    JOIN zones z ON t.PUlocationID = z.LocationID
""")

In [77]:
# Count occurrences of each zone
zone_counts = joined_df.groupBy("Zone").count()

In [78]:
zone_counts.show()

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|           Homecrest| 1295|
|Governor's Island...|    2|
|              Corona| 7175|
|    Bensonhurst West| 1880|
|         Westerleigh| 1317|
|      Newark Airport| 2532|
|Charleston/Totten...| 2533|
|          Douglaston|  291|
|      Pelham Parkway| 1522|
|East Concourse/Co...| 3294|
|          Mount Hope| 3973|
|         Marble Hill|  116|
|           Rego Park| 1697|
|Upper East Side S...| 2358|
|       Dyker Heights|  690|
|Heartland Village...| 1054|
|   Kew Gardens Hills|  906|
|     Jackson Heights|10952|
|             Bayside| 2547|
|TriBeCa/Civic Center| 1918|
+--------------------+-----+
only showing top 20 rows



In [79]:
# Find the least frequent pickup location zone
least_frequent_zone = zone_counts.orderBy("count").first()["Zone"]

In [80]:
print(f"The LEAST frequent pickup location zone is: {least_frequent_zone}")

The LEAST frequent pickup location zone is: Jamaica Bay
